<a href="https://colab.research.google.com/github/dmceballosg/PruebaComfama/blob/main/PruebaAnalistaDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Punto 1
===

In [66]:
# Recuerde cargar los datos en la carpeta temporal, en formato xlsx, o modifique el código para cumplir 
# Con el formato y la ruta que usted desea 

import pandas as pd
import numpy as np

def readData(path):
  return pd.read_excel(path)

data = readData("/tmp/Prueba_Tecnica_Datos.xlsx")

In [67]:
pd.set_option('display.max_columns', None) 
print("\nDimensión de la data:\n")
print(data.shape)
print("\nBreve resumen de la data:\n")
print(data.head(10))
print("\nMedidas estadisticas de dispersión y tendencia central para caracteristicas cuantitativos:\n")
print(data.describe())



Dimensión de la data:

(20011, 22)

Breve resumen de la data:

   IdUsuario                             Agencia Ciudad/De/Residencia  \
0   306125.0                 COMFAMASEDEPEDREGAL             MEDELLÍN   
1   103291.0                  COMFAMABARRIOVELEZ             APARTADÓ   
2   306120.0  COMFAMACARRERA44CLAUSTROSANIGNACIO                BELLO   
3   311871.0                 COMFAMAPUERTOBERRIO        PUERTO BERRÍO   
4     8672.0  COMFAMACARRERA44CLAUSTROSANIGNACIO             MEDELLÍN   
5   225667.0  COMFAMACARRERA44CLAUSTROSANIGNACIO            GIRARDOTA   
6   242920.0  COMFAMACARRERA44CLAUSTROSANIGNACIO             MEDELLÍN   
7   102762.0                     EstrategiaMóvil                ANDES   
8     8687.0  COMFAMACARRERA44CLAUSTROSANIGNACIO                BELLO   
9     8619.0  COMFAMACARRERA44CLAUSTROSANIGNACIO             MEDELLÍN   

              FechaRegistro Canal////DeRegistro     fecha/Nacimiento  \
0                     43411        Autoregistro  1998-04-14 

In [68]:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
# from pandas_profiling import ProfileReport    
# profile = ProfileReport(data, title="Monitoria Profiling Report")
# profile.to_notebook_iframe()

In [69]:
# Determinación de datos faltantes por columans: 
def countMissing(dataSet):
  counted = pd.DataFrame(dataSet.isnull().sum())
  counted.columns = ['Valor']
  return counted

def countMissingPercent(dataset):
  counted = countMissing(dataset)
  counted = pd.DataFrame(counted).transpose();
  counted = counted.transform(lambda x : (x/dataset.shape[0])*100)
  counted = counted.transpose()
  counted.columns = ['Valor']
  return counted
 
print(countMissing(data))
print(countMissingPercent(data))

                                 Valor
IdUsuario                           10
Agencia                            347
Ciudad/De/Residencia                 0
FechaRegistro                        0
Canal////DeRegistro                  0
fecha/Nacimiento                     1
Edad                                 1
Genero                               0
PorcentajeHojaVida                   0
Fecha/Actualizacion                  5
NiveldeEstudio                    2286
TituloObtenido                    2626
Programa///degobierno            14281
CondicionesEspeciales            19556
SituacionLaboral                     2
PrestadorAnterior                16301
FechaCambioPrestador             16303
VeredaLocalidad///CentroPoblado  19315
PerteneceA                         595
AspiracionSalarial//               942
Ocupacion                         3901
Estado                               2
                                     Valor
IdUsuario                         0.049973
Agencia          

In [70]:
def optionalColumns(dataset):
  totalData = countMissingPercent(dataset)
  totalData_onlyOptional = pd.DataFrame(totalData.copy())
  totalData_onlyOptional = totalData_onlyOptional[totalData_onlyOptional["Valor"] > 50]
  return totalData_onlyOptional

print(optionalColumns(data))

                                     Valor
Programa///degobierno            71.365749
CondicionesEspeciales            97.726251
PrestadorAnterior                81.460197
FechaCambioPrestador             81.470191
VeredaLocalidad///CentroPoblado  96.521913


In [71]:
def getOnlyOptionalColumns(dataset):
  optionals = pd.DataFrame(optionalColumns(dataset)).reset_index()
  return list(optionals["index"])

def getOnlyRequiredColumns(dataset):
  optionals = set(getOnlyOptionalColumns(dataset))
  allColumns = set(list(dataset.columns))
  required = list(allColumns - optionals)
  return required

def deleteMissing(dataset, column):
  dataset = dataset[dataset[column].notna()]
  return dataset

def deleteAllMissing(dataset, requiredColumns):
  for column in requiredColumns:
    dataset = deleteMissing(dataset, column)

  return dataset

required = getOnlyRequiredColumns(data);
data = deleteAllMissing(data,required)
data.shape

(14877, 22)

In [72]:
def replaceNullValue(dataset,column):
  dataNA = dataset.copy()
  dataNA[column].fillna(column, inplace = True)
  return dataNA 

def replaceAllNullValue(dataset,optionalColumns):
  for column in optionalColumns:
    dataset = replaceNullValue(dataset, column)
  return dataset

optional = getOnlyOptionalColumns(data)
data = replaceAllNullValue(data,optional)

print(countMissing(data))

                                 Valor
IdUsuario                            0
Agencia                              0
Ciudad/De/Residencia                 0
FechaRegistro                        0
Canal////DeRegistro                  0
fecha/Nacimiento                     0
Edad                                 0
Genero                               0
PorcentajeHojaVida                   0
Fecha/Actualizacion                  0
NiveldeEstudio                       0
TituloObtenido                       0
Programa///degobierno                0
CondicionesEspeciales                0
SituacionLaboral                     0
PrestadorAnterior                    0
FechaCambioPrestador                 0
VeredaLocalidad///CentroPoblado      0
PerteneceA                           0
AspiracionSalarial//                 0
Ocupacion                            0
Estado                               0


In [73]:
def isNumber(dataset,column,typeComparation):
  dataset["is_numeric"] = dataset[column].apply(np.isreal)
  hasInvalidType = dataset[dataset["is_numeric"] == typeComparation]
  hastInvalidType = hasInvalidType.drop('is_numeric', axis=1)
  return hasInvalidType

def isAllNumber(dataset, columns, typeComparation):
  for column in columns:
    hasInvalidType = isNumber(dataset, column, typeComparation)
    if hasInvalidType.shape[0] > 0 :
      print(hasInvalidType[column])

numericColumns = ['IdUsuario','Edad','PorcentajeHojaVida']

print(isAllNumber(data, numericColumns, False))

None


In [74]:
def validateNumericRange(dataset,columnsRange):
  print(columnsRange[0] + ": ",end = '')
  print(list(dataset[(dataset[columnsRange[0]] < columnsRange[1][0]) | (dataset[columnsRange[0]] > columnsRange[1][1])][columnsRange[0]]))
  dataset = dataset[(dataset[columnsRange[0]] >= columnsRange[1][0]) & (dataset[columnsRange[0]] <= columnsRange[1][1])]
  return dataset

def validateAllNumericRange(dataset,columnsRange):
  for column in columnsRange:
    dataset = validateNumericRange(dataset,column)
  return dataset 

numericColumnsRange = [('IdUsuario',[0,np.inf]),('Edad',[18,80]),('PorcentajeHojaVida',[0,100])]

data = validateAllNumericRange(data,numericColumnsRange)

print(data.shape)


IdUsuario: [-34.0, -1.0]
Edad: [-1778.7232876712328, 17.27123287671233, 17.44109589041096, 80.35068493150685, 16.375342465753423, 99.8986301369863]
PorcentajeHojaVida: [250, 980, 120, -50, 560, 256]
(14863, 23)


In [75]:
textColumns = ['Agencia','Ciudad/De/Residencia','Canal////DeRegistro','Genero','NiveldeEstudio','TituloObtenido','Programa///degobierno','CondicionesEspeciales','SituacionLaboral',
                'PrestadorAnterior','VeredaLocalidad///CentroPoblado','PerteneceA','AspiracionSalarial//','Ocupacion','Estado'
              ]

print(isAllNumber(data, textColumns, True))

6086     2017-06-01 00:00:00
8897                       9
10011                      6
11880                      3
12866                     11
14320                     10
14476                      5
16885                     11
19609                     11
Name: TituloObtenido, dtype: object
9927     3044856868
17495      43307100
Name: Ocupacion, dtype: object
None


In [76]:
def onlyText(dataset,column,typeComparation):
  dataset["is_numeric"] = dataset[column].apply(np.isreal)
  hasInvalidType = dataset[dataset["is_numeric"] != typeComparation]
  hastInvalidType = hasInvalidType.drop('is_numeric', axis=1)
  return hasInvalidType

def isAllOnlyText(dataset, columns, typeComparation):
  for column in columns:
    dataset = onlyText(dataset, column, typeComparation)
  return dataset

data = isAllOnlyText(data, textColumns, True)
data.shape


(14852, 23)

In [77]:
from datetime import datetime
import xlrd

def formatDate(dataset, column):
  df = dataset.copy()
  df["isNumber"] = df[column].apply(np.isreal)
  df["fechaCopy"] = df[column].copy()
  df["fechaCopy"] =  pd.to_datetime(df['fechaCopy'], errors="coerce")
  df["dataTimeFinal"] = df[column].apply(lambda x: xlrd.xldate_as_datetime(x,0) if str(x).isnumeric() else x)
  df.loc[df["isNumber"] == True, 'total'] = df["dataTimeFinal"]
  df.loc[df["isNumber"] == False, 'total'] = df["fechaCopy"]
  df = df.drop('isNumber', axis=1)
  df = df.drop('fechaCopy', axis=1)
  return df["total"]

def formatAllDate(dataset, columns):
  for column in columns:
    dataset[column] = formatDate(dataset, column);
  return dataset
  
dataColumns = ["FechaRegistro","fecha/Nacimiento","Fecha/Actualizacion"]
data = formatAllDate(data, dataColumns)
print(data[dataColumns])



             FechaRegistro fecha/Nacimiento  Fecha/Actualizacion
0      2018-11-07 00:00:00       1998-04-14  2018-11-07 00:00:00
1      2016-01-28 00:00:00       1993-07-18  2020-11-13 00:00:00
2      2016-03-03 08:21:58       1995-08-21  2016-04-19 10:37:06
3      2017-07-11 15:27:47       1990-09-17  2020-12-15 16:01:43
4      2016-10-06 15:44:17       1966-01-14  2017-10-07 09:50:53
...                    ...              ...                  ...
20006  2014-01-30 12:58:01       1984-05-08  2019-11-19 12:45:12
20007  2017-11-14 12:40:29       1984-08-17  2017-11-15 14:51:46
20008  2017-08-23 09:57:43       1980-03-01  2018-11-05 15:08:27
20009  2017-08-17 10:23:14       1990-09-15  2017-11-09 09:42:08
20010  2014-10-14 08:20:45       1960-01-18  2014-10-27 23:01:53

[14852 rows x 3 columns]


In [78]:
def convertDataOptional(dataset):
  dataset.loc[dataset["FechaCambioPrestador"] == "FechaCambioPrestador", 'FechaCambioPrestador'] = "31/12/2000 11:59:00 p.m."
  dataset["FechaCambioPrestador"]
  return dataset["FechaCambioPrestador"]

data["FechaCambioPrestador"] = convertDataOptional(data)
data = formatAllDate(data, ["FechaCambioPrestador"])
data["FechaCambioPrestador"]


0        2000-12-31 23:59:00
1        2016-07-22 00:00:00
2        2000-12-31 23:59:00
3        2018-01-30 09:18:13
4        2000-12-31 23:59:00
                ...         
20006    2000-12-31 23:59:00
20007    2000-12-31 23:59:00
20008    2000-12-31 23:59:00
20009    2000-12-31 23:59:00
20010    2000-12-31 23:59:00
Name: FechaCambioPrestador, Length: 14852, dtype: object

In [79]:
from datetime import date

def validateDataRange(dataset,columnsRange):
  print(columnsRange[0] + ": ",end = '')
  print(list(dataset[(dataset[columnsRange[0]] <  pd.to_datetime(columnsRange[1][0])) | (dataset[columnsRange[0]] > pd.to_datetime(columnsRange[1][1]))][columnsRange[0]]))
  dataset = dataset[(dataset[columnsRange[0]] >= pd.to_datetime(columnsRange[1][0])) & (dataset[columnsRange[0]] <= pd.to_datetime(columnsRange[1][1]))]
  return dataset

def validateAllDataRange(dataset,columnsRange):
  for column in columnsRange:
    dataset = validateDataRange(dataset,column)
  return dataset 

today = str(date.today())
DateColumnsRange = [('FechaRegistro',['1960-01-01', today]),('fecha/Nacimiento',['1960-01-01', today]),('Fecha/Actualizacion',['2000-01-01', today])]

data1 = data.copy();

validateAllDataRange(data1, DateColumnsRange)
# print(DateColumnsRange)


FechaRegistro: []
fecha/Nacimiento: [Timestamp('1951-03-13 00:00:00'), Timestamp('1952-05-16 00:00:00'), Timestamp('1957-01-03 00:00:00'), Timestamp('1958-03-18 00:00:00'), Timestamp('1952-02-04 00:00:00'), Timestamp('1958-09-22 00:00:00'), Timestamp('1959-04-18 00:00:00'), Timestamp('1955-07-03 00:00:00'), Timestamp('1958-03-06 00:00:00'), Timestamp('1951-01-18 00:00:00'), Timestamp('1958-04-28 00:00:00'), Timestamp('1956-07-28 00:00:00'), Timestamp('1959-05-12 00:00:00'), Timestamp('1959-02-10 00:00:00'), Timestamp('1959-07-20 00:00:00'), Timestamp('1958-05-31 00:00:00'), Timestamp('1956-06-13 00:00:00'), Timestamp('1956-08-17 00:00:00'), Timestamp('1957-01-11 00:00:00'), Timestamp('1952-06-10 00:00:00'), Timestamp('1954-04-13 00:00:00'), Timestamp('1945-07-20 00:00:00'), Timestamp('1957-02-19 00:00:00'), Timestamp('1957-03-27 00:00:00'), Timestamp('1957-06-10 00:00:00'), Timestamp('1958-05-29 00:00:00'), Timestamp('1958-04-27 00:00:00'), Timestamp('1959-07-13 00:00:00'), Timestamp('

,IdUsuario,Agencia,Ciudad/De/Residencia,FechaRegistro,Canal////DeRegistro,fecha/Nacimiento,Edad,Genero,PorcentajeHojaVida,Fecha/Actualizacion,NiveldeEstudio,TituloObtenido,Programa///degobierno,CondicionesEspeciales,SituacionLaboral,PrestadorAnterior,FechaCambioPrestador,VeredaLocalidad///CentroPoblado,PerteneceA,AspiracionSalarial//,Ocupacion,Estado,is_numeric
0,306125.0,COMFAMASEDEPEDREGAL,MEDELLÍN,2018-11-07 00:00:00,Autoregistro,1998-04-14,23.583562,F,100,2018-11-07 00:00:00,Tecnológica,tecnología en gestión empresarial,Programa///degobierno,CondicionesEspeciales,Empleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,Auxiliar Operativo,Inactivo x Tiempo,False
1,103291.0,COMFAMABARRIOVELEZ,APARTADÓ,2016-01-28 00:00:00,Autoregistro,1993-07-18,28.326027,F,100,2020-11-13 00:00:00,Universitaria,Adminitradora en Salud,Víctimas del conflicto armado,CondicionesEspeciales,Desempleado,CAJA DE COMPENSACIÓN FAMILIAR COMFENALCO ANTIO...,2016-07-22 00:00:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,Ejecutiva comercial,Activo,False
2,306120.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,BELLO,2016-03-03 08:21:58,Agencia,1995-08-21,26.232877,F,100,2016-04-19 10:37:06,Técnica Laboral,Auxiliar Administrativo en salud,40 Mil Primeros Empleos,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,Practicante,Inactivo x Tiempo,False
3,311871.0,COMFAMAPUERTOBERRIO,PUERTO BERRÍO,2017-07-11 15:27:47,Agencia,1990-09-17,31.161644,F,100,2020-12-15 16:01:43,Media(10-13),Bachiller Académico,Víctimas del conflicto armado,CondicionesEspeciales,Primer Empleo,CAJA DE COMPENSACIÓN FAMILIAR COMFENALCO ANTIO...,2018-01-30 09:18:13,VeredaLocalidad///CentroPoblado,Urbano,A convenir,Asesor servicio al cliente,Activo,False
4,8672.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2016-10-06 15:44:17,Autoregistro,1966-01-14,55.852055,F,100,2017-10-07 09:50:53,Técnica Laboral,TECNICA EN GESTION HUMANA,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,PRACTICANTE,Inactivo x Tiempo,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20006,6458.0,COMFAMABARRIOVELEZ,CHIGORODÓ,2014-01-30 12:58:01,Autoregistro,1984-05-08,37.526027,F,100,2019-11-19 12:45:12,Media(10-13),BACHILLER,Víctimas del conflicto armado,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,Calificar labores oficios varios,Inactivo x Tiempo,False
20007,8777.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2017-11-14 12:40:29,Autoregistro,1984-08-17,37.249315,F,100,2017-11-15 14:51:46,Media(10-13),BACHILLER,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,AUXILIAR DE CREDITO Y RECEPCION,Inactivo x Tiempo,False
20008,1657.0,COMFAMAELBAGRE,EL BAGRE,2017-08-23 09:57:43,Agencia,1980-03-01,41.715068,M,100,2018-11-05 15:08:27,Media(10-13),Bachiller academico,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Rural,A convenir,Servicios Generales,Inactivo x Tiempo,False
20009,104098.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2017-08-17 10:23:14,Agencia,1990-09-15,31.167123,F,100,2017-11-09 09:42:08,Media(10-13),Técnica Normalista Superior,Fondo de Oportunidades para el Empleo,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,Docente de Básica Primaria,Activo,False


In [80]:
def findExactMatch(dataset):
  return dataset[dataset.duplicated(keep='first')]
  
findExactMatch(data).shape

(0, 23)

In [81]:

def repetedThreshold(dataset,threshold):
  data2 = dataset.sort_values(['IdUsuario','FechaRegistro'])
  data3 =  data2.groupby('IdUsuario')
  return data3.filter(lambda x: len(x) > 15)

data2 = repetedThreshold(data, 15)
data2

,IdUsuario,Agencia,Ciudad/De/Residencia,FechaRegistro,Canal////DeRegistro,fecha/Nacimiento,Edad,Genero,PorcentajeHojaVida,Fecha/Actualizacion,NiveldeEstudio,TituloObtenido,Programa///degobierno,CondicionesEspeciales,SituacionLaboral,PrestadorAnterior,FechaCambioPrestador,VeredaLocalidad///CentroPoblado,PerteneceA,AspiracionSalarial//,Ocupacion,Estado,is_numeric
3469,4313.0,COMFAMALOSMOLINOSMEDELLÍN,MEDELLÍN,2013-06-28 14:57:10,Agencia,1982-04-30,39.550685,F,100,2021-12-03 18:55:13,Técnica Laboral,tecnica en diseño grafico,Programa///degobierno,CondicionesEspeciales,Desempleado,ALCALDÍA DE MEDELLÍN,2019-09-17 17:22:50,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,Asesora comercial,Activo,False
8048,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2014-05-06 10:53:09,Autoregistro,1961-09-25,60.158904,F,100,2020-02-03 14:23:14,Media(10-13),Bachiller académico,Programa///degobierno,CondicionesEspeciales,Desempleado,CAJA DE COMPENSACIÓN FAMILIAR COMFENALCO ANTIO...,2020-02-19 10:04:21,VeredaLocalidad///CentroPoblado,Urbano,A convenir,Cuidador adulto mayor,Inactivo x Tiempo,False
2646,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2014-08-14 11:28:28,Autoregistro,1985-07-27,36.306849,F,100,2021-02-28 19:26:36,Universitaria,ABOGADO,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,auxiliar administrativa,Activo,False
11780,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,BELLO,2015-09-10 11:11:44,Autoregistro,1976-01-21,45.827397,F,100,2018-05-17 17:46:23,Tecnológica,NORMALISTA SUPERIOR,"Fondo de Oportunidades para el Empleo, Víctima...",CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,DOCENTE,Inactivo x Tiempo,False
8754,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2016-07-10 20:01:17,Autoregistro,1966-03-03,55.720548,F,100,2020-06-04 07:50:26,Básica Secundaria(6-9),Académico,Fondo de Oportunidades para el Empleo,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 SMMLV,madre comunitaria,Inactivo x Tiempo,False
19784,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2016-07-18 12:52:52,Autoregistro,1981-09-26,40.142466,F,100,2016-07-18 13:25:23,Básica Secundaria(6-9),bachiller,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,asesora,Inactivo x Tiempo,False
1913,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2016-09-03 10:08:16,Autoregistro,1975-06-21,46.413699,F,100,2017-06-06 10:08:46,Técnica Laboral,MERCADOLOGA,Fondo de Oportunidades para el Empleo,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,2 a 4 SMMLV,ASESORA COMERCIAL,Inactivo x Tiempo,False
19390,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,ARBOLETES,2017-02-02 12:47:09,Autoregistro,1972-02-27,49.728767,F,100,2017-02-02 13:07:08,Técnica Laboral,Técnico profesional auxiliar enfermeria,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,Auxiliar de enfermeria,Inactivo x Tiempo,False
4487,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2017-03-05 21:41:27,Autoregistro,1968-07-26,53.320548,F,100,2017-05-05 14:57:55,Especialización,ESPECIALISTA EN GERENCIA DE MERCADEO,Programa///degobierno,CondicionesEspeciales,Empleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,6 a 9 SMMLV,KAM NACIONAL,Inactivo x Tiempo,False
7231,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2017-06-27 14:54:55,Autoregistro,1973-04-20,48.583562,F,100,2018-02-02 08:00:36,Tecnológica,TECNOLOGIA EN GESTION ADMINISTRATIVA,Programa///degobierno,CondicionesEspeciales,Empleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,1 a 2 SMMLV,AUXILIAR ADMINISTRATIVA EN S

In [82]:
def dropDuplicated(dataset):
   data2 = dataset.sort_values(['IdUsuario','FechaRegistro'])
   data2["Id"] = dataset["IdUsuario"]
   data2 = data2.drop_duplicates(['Id'], keep='last').set_index('Id')
   return data2

data = dropDuplicated(data)
data[data["IdUsuario"] == 4313]

,IdUsuario,Agencia,Ciudad/De/Residencia,FechaRegistro,Canal////DeRegistro,fecha/Nacimiento,Edad,Genero,PorcentajeHojaVida,Fecha/Actualizacion,NiveldeEstudio,TituloObtenido,Programa///degobierno,CondicionesEspeciales,SituacionLaboral,PrestadorAnterior,FechaCambioPrestador,VeredaLocalidad///CentroPoblado,PerteneceA,AspiracionSalarial//,Ocupacion,Estado,is_numeric
Id,,,,,,,,,,,,,,,,,,,,,,,
4313.0,4313.0,COMFAMACARRERA44CLAUSTROSANIGNACIO,MEDELLÍN,2020-01-28 10:30:40,Agencia,1982-02-24,39.728767,F,100,2020-08-02 09:43:18,Media(10-13),MEDIA,Programa///degobierno,CondicionesEspeciales,Desempleado,PrestadorAnterior,2000-12-31 23:59:00,VeredaLocalidad///CentroPoblado,Urbano,A convenir,OPERARIA DE MUESTRAS,Activo,False


In [83]:
data.shape

(7956, 23)

In [84]:
def redundantColumn(dataset, column):
   return dataset.drop(column, axis=1)

data = redundantColumn(data, 'Edad')
data = redundantColumn(data, 'is_numeric') 
data.shape

(7956, 21)

In [88]:
data.to_csv("/tmp/dataFinal.csv", index=False)